# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
!echo $CUDA_VISIBLE_DEVICES

2


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'ArcHead' # ''ArcHead', CosHead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception
#     But if you are trying to use NasNet, please check this issue first: https://github.com/keras-team/keras-applications/issues/78
#         We manually download the weight file and explicitly load it in models.py file
# Backbones w/o pretrained weights:
#     MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
backbone_type = 'InceptionResNetV2' 
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print(train_size)
steps_per_epoch = train_size // batch_size
print(steps_per_epoch)
val_size = dataset_len - train_size
print(val_size)
validation_steps = val_size // batch_size
print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

4658122
36391
1164531
9097


In [4]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.summary()

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

219062272/219055592 [==============================] - 3s 0us/step
Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
inception_resnet_v2 (Functional (None, 2, 2, 1536)   54336736    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          3154432     inception_resnet_v2[0][0]        
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
___________________

In [ ]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
#                               patience=0.1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 2:17:41 - loss: 46.2284
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_45.90513229370117.ckpt
 2000/36391 [>.............................] - ETA: 2:11:57 - loss: 45.8746
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_45.135643005371094.ckpt
 3000/36391 [=>............................] - ETA: 2:07:19 - loss: 45.5020
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_44.3824577331543.ckpt
 4000/36391 [==>...........................] - ETA: 2:02:38 - loss: 45.1306
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_43.65995788574219.ckpt
 5000/36391 [===>............

35000/36391 [===========================>..] - ETA: 5:02 - loss: 36.3790
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_28.874958038330078.ckpt
36000/36391 [============================>.] - ETA: 1:25 - loss: 36.1666
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_1_l_28.59041404724121.ckpt
36391/36391 [==============================] - 8533s 233ms/step - loss: 36.0844 - val_loss: 18.1403
Epoch 2/20
  609/36391 [..............................] - ETA: 2:09:49 - loss: 18.3476
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_2_l_18.277145385742188.ckpt
 1609/36391 [>.............................] - ETA: 2:08:14 - loss: 18.2895
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1

32609/36391 [=========================>....] - ETA: 15:21 - loss: 16.8760
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_2_l_15.770243644714355.ckpt
33609/36391 [==========================>...] - ETA: 11:15 - loss: 16.8423
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_2_l_15.717179298400879.ckpt
34609/36391 [===========================>..] - ETA: 7:12 - loss: 16.8090
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_2_l_15.665343284606934.ckpt
35609/36391 [============================>.] - ETA: 3:09 - loss: 16.7762
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_2_l_15.61483097076416.ckpt
36391/36391 [==============================] - 

30218/36391 [=======================>......] - ETA: 22:54 - loss: 13.3063
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_3_l_12.885156631469727.ckpt
31218/36391 [========================>.....] - ETA: 19:11 - loss: 13.2924
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_3_l_12.86009407043457.ckpt
32218/36391 [=========================>....] - ETA: 15:28 - loss: 13.2786
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_3_l_12.8345308303833.ckpt
33218/36391 [==========================>...] - ETA: 11:45 - loss: 13.2649
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_3_l_12.809591293334961.ckpt
34218/36391 [===========================>..] - 

27827/36391 [=====================>........] - ETA: 32:08 - loss: 11.6467
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_4_l_11.406905174255371.ckpt
28827/36391 [======================>.......] - ETA: 28:23 - loss: 11.6381
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_4_l_11.391752243041992.ckpt
29827/36391 [=======================>......] - ETA: 24:38 - loss: 11.6297
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_4_l_11.378826141357422.ckpt
30827/36391 [========================>.....] - ETA: 20:53 - loss: 11.6213- ETA: 20:55 - loss:
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_4_l_11.363432884216309.ckpt
31827/36391 [===========

...

23045/36391 [=================>............] - ETA: 50:02 - loss: 9.8882
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_6_l_9.773768424987793.ckpt
24045/36391 [==================>...........] - ETA: 46:19 - loss: 9.8833
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_6_l_9.76669979095459.ckpt
25045/36391 [===================>..........] - ETA: 42:32 - loss: 9.8785
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_6_l_9.758171081542969.ckpt
26045/36391 [====================>.........] - ETA: 38:46 - loss: 9.8736
Epoch 00006: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_6_l_9.748791694641113.ckpt
27045/36391 [=====================>........] - ETA: 3

20654/36391 [================>.............] - ETA: 56:59 - loss: 9.2665
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_7_l_9.187722206115723.ckpt
21654/36391 [================>.............] - ETA: 53:22 - loss: 9.2628
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_7_l_9.183362007141113.ckpt
22654/36391 [=================>............] - ETA: 49:44 - loss: 9.2592
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_7_l_9.184002876281738.ckpt
23654/36391 [==================>...........] - ETA: 46:06 - loss: 9.2560
Epoch 00007: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_7_l_9.182215690612793.ckpt
24654/36391 [===================>..........] - ETA: 

18263/36391 [==============>...............] - ETA: 1:04:48 - loss: 8.8413
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_8_l_8.765859603881836.ckpt
19263/36391 [==============>...............] - ETA: 1:01:14 - loss: 8.8371
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_8_l_8.755838394165039.ckpt
20263/36391 [===============>..............] - ETA: 57:43 - loss: 8.8328
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_8_l_8.746715545654297.ckpt
21263/36391 [================>.............] - ETA: 54:09 - loss: 8.8290
Epoch 00008: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_8_l_8.750197410583496.ckpt
22263/36391 [=================>............] - E

15872/36391 [============>.................] - ETA: 1:21:30 - loss: 8.5590
Epoch 00009: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_9_l_8.467183113098145.ckpt
16872/36391 [============>.................] - ETA: 1:17:58 - loss: 8.5531
Epoch 00009: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_9_l_8.453076362609863.ckpt
17872/36391 [=============>................] - ETA: 1:14:22 - loss: 8.5472
Epoch 00009: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_9_l_8.443682670593262.ckpt
18872/36391 [==============>...............] - ETA: 1:10:38 - loss: 8.5413
Epoch 00009: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_9_l_8.428133964538574.ckpt
19872/36391 [===============>..............]

13481/36391 [==========>...................] - ETA: 1:38:52 - loss: 7.9952
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_10_l_7.9355854988098145.ckpt
14481/36391 [==========>...................] - ETA: 1:34:28 - loss: 7.9910
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_10_l_7.9296040534973145.ckpt
15481/36391 [===========>..................] - ETA: 1:30:08 - loss: 7.9868
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_10_l_7.922982692718506.ckpt
16481/36391 [============>.................] - ETA: 1:25:46 - loss: 7.9827
Epoch 00010: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_InceptionResNetV2_ArcHead_check/SGD/e_10_l_7.916577339172363.ckpt
17481/36391 [=============>...........

### Resume training with latest checkpoint

In [12]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"+version+"/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_CosHead_check/SGD/e_1_l_47.875667572021484.ckpt


In [13]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_"+backbone_type+"_"+head_type+"_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413024     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [14]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
    3/36391 [..............................] - ETA: 4:17:41 - loss: 47.6510 

KeyboardInterrupt: 